In [1]:
pip install groq

In [2]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)


chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are essential for various natural language processing (NLP) applications, and their importance can be understood from several perspectives:

1. **Efficient Processing**: Fast language models enable rapid processing of large volumes of text data, which is crucial for real-time applications such as chatbots, virtual assistants, and sentiment analysis. They can handle multiple requests and responses quickly, providing a seamless user experience.
2. **Scalability**: Fast language models can handle a large number of users and requests simultaneously, making them ideal for applications with high traffic or usage. This scalability ensures that the model can handle increased loads without compromising performance.
3. **Improved User Experience**: Fast language models can respond to user queries quickly, which improves the overall user experience. In applications like language translation, sentiment analysis, or text summarization, faster models can provide results in real-

In [3]:

import openai

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

In [4]:
from groq import Groq
client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)
completion = client.chat.completions.create(
    model="llama-3.2-11b-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What's in this image?"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/f/f2/LPU-v1-die.jpg"
                    }
                }
            ]
        }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=False,
    stop=None,
)

print(completion.choices[0].message)


ChatCompletionMessage(content="This image could be the brain of a supercomputer or the motherboard of a machine. But without more information, we can't say what it is.", role='assistant', function_call=None, tool_calls=None)


In [23]:
from groq import Groq
import base64


# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "/content/download.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                    },
                },
            ],
        }
    ],
    model="llama-3.2-11b-vision-preview",
)

print(chat_completion.choices[0].message.content)

The image depicts a serene and majestic ocean scene, with the sun peeking through the clouds on the horizon. The ocean's calmness is depicted by the gentle waves and the deep blue color of the water. The sky is a mix of dark and light clouds, with the sun shining through, creating a beautiful contrast of colors. The overall atmosphere of the image is one of tranquility and peace, evoking feelings of relaxation and wonder.


In [5]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [6]:
import PyPDF2
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
import requests
import json

def convert_pdf_to_vector_db(pdf_paths):
    text = ""
    for pdf_path in pdf_paths:
        pdf_reader = PyPDF2.PdfReader(pdf_path)
        for page in pdf_reader.pages:
            text += page.extract_text()

    text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200, length_function=len)
    chunks = text_splitter.split_text(text)

    embeddings = []
    for chunk in chunks:
        response = requests.post(
            "https://api.groq.io/v1/embed",
            headers={"Authorization": userdata.get("GROQ_API_KEY")},
            json={"text": chunk}
        )
        if response.status_code == 200:
            embedding = response.json()["embedding"]
            embeddings.append(embedding)
        else:
            print(f"Error generating embedding for text: {chunk}")

    return FAISS.from_texts(chunks, embeddings)

# Example usage:
pdf_paths = ['path/to/pdf1.pdf', 'path/to/pdf2.pdf', ...]
vector_db = convert_pdf_to_vector_db(pdf_paths)

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/pdf1.pdf'

In [7]:
pip install faiss-cpu

In [10]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [28]:
import PyPDF2
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.docstore.document import Document
from groq import Groq


def build_faiss_index(pdf_paths, output_dir="faiss_index"):
    """
    Build and save a FAISS vector store from a list of PDF paths.

    Args:
        pdf_paths (list): List of paths to PDF files.
        output_dir (str): Directory to save the FAISS index.
    """
    text = ""
    for pdf_path in pdf_paths:
        pdf_reader = PyPDF2.PdfReader(pdf_path)
        for page in pdf_reader.pages:
            text += page.extract_text()

    text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200, length_function=len)
    chunks = text_splitter.split_text(text)

    docs = [Document(page_content=chunk) for chunk in chunks]
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

    faiss_db = FAISS.from_documents(docs, embedding_function)
    faiss_db.save_local(output_dir)


def get_relevant_excerpts(user_question, faiss_db):
    """
    Retrieve the most relevant excerpts from FAISS vector store based on the user's question.

    Args:
        user_question (str): The user's question.
        faiss_db (FAISS): FAISS vector store.

    Returns:
        str: Combined relevant excerpts.
    """
    relevant_docs = faiss_db.similarity_search(user_question, k=3)
    relevant_excerpts = '\n\n------------------------------------------------------\n\n'.join(
        [doc.page_content for doc in relevant_docs]
    )
    return relevant_excerpts


def presidential_speech_chat_completion(client, model, user_question, relevant_excerpts):
    """
    Generate a response to the user's question using a pre-trained model.

    Args:
        client (Groq): Groq client.
        model (str): Pre-trained model name.
        user_question (str): User's question.
        relevant_excerpts (str): Relevant excerpts from speeches.

    Returns:
        str: Response to the user's question.
    """
    system_prompt = '''
    You are a presidential historian. Given the user's question and relevant excerpts from
    presidential speeches, answer the question by including direct quotes from presidential speeches.
    When using a quote, cite the speech that it was from (ignoring the chunk).
    '''

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"User Question: {user_question}\n\nRelevant Speech Excerpt(s):\n\n{relevant_excerpts}"}
        ],
        model=model
    )

    response = chat_completion.choices[0].message.content
    return response


def main():
    """
    Main function to run the application using FAISS instead of Pinecone.
    """
    model = 'llama3-8b-8192'
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

    # Load FAISS database
    # Load FAISS database
    faiss_db = FAISS.load_local("faiss_index", embedding_function, allow_dangerous_deserialization=True)

    # Initialize Groq client
    groq_api_key = os.environ.get('GROQ_API_KEY')
    client = Groq(api_key=groq_api_key)

    # Display application introduction
    print("Presidential Speeches RAG")
    print("""
    Welcome! Ask questions about U.S. presidents, like "What were George Washington's views on democracy?" or "What did Abraham Lincoln say about national unity?". The app matches your question to relevant excerpts from presidential speeches and generates a response using a pre-trained model.
    """)

    while True:
        user_question = input("Ask a question about a US president: ")

        if user_question:
            relevant_excerpts = get_relevant_excerpts(user_question, faiss_db)
            response = presidential_speech_chat_completion(client, model, user_question, relevant_excerpts)
            print(response)


if __name__ == "__main__":
    # Example usage with your own PDF files
    pdf_paths = ['/content/Short write up BDM Capstone Project.pdf']
    build_faiss_index(pdf_paths)
    main()


<ipython-input-28-21fef7f5f18f>:27: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


KeyboardInterrupt: 

In [14]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 19.3 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.155.0
    Uninstalling google-api-python-client-2.155.0:
      Successfully uninstalled google-api-python-client-2.155.0


In [29]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import os.path

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/classroom.courses.readonly"]

def main():
    """Shows basic usage of the Classroom API.
    Prints the names of the first 10 courses the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            # Run the local server with a specific redirect URI
            creds = flow.run_local_server(
                port=0,
                open_browser=False,
                authorization_prompt_message="Please visit this URL to authorize: {url}"
            )
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("classroom", "v1", credentials=creds)

        # Call the Classroom API
        results = service.courses().list(pageSize=10).execute()
        courses = results.get("courses", [])

        if not courses:
            print("No courses found.")
            return
        # Prints the names of the first 10 courses.
        print("Courses:")
        for course in courses:
            print(course["name"])

    except HttpError as error:
        print(f"An error occurred: {error}")

if __name__ == "__main__":
    main()


Courses:
Test


In [30]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [35]:
from google.colab import auth
from googleapiclient.discovery import build
import google.auth
from google.auth.transport.requests import Request

# Authenticate the user with the required scopes
auth.authenticate_user()

# Specify the required scopes
SCOPES = [
    'https://www.googleapis.com/auth/classroom.courses.readonly',
    'https://www.googleapis.com/auth/classroom.coursework.me',
    'https://www.googleapis.com/auth/classroom.submissions.readonly'
]

# Authenticate and create the service with the specified scopes
creds, _ = google.auth.default(scopes=SCOPES)

# If credentials are invalid or expired, request new credentials
if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())

service = build('classroom', 'v1', credentials=creds)

# Fetch all courses
def list_courses():
    courses = []
    results = service.courses().list().execute()
    courses = results.get('courses', [])
    return courses

# Fetch assignments for a course
def list_assignments(course_id):
    assignments = []
    coursework_results = service.courses().courseWork().list(courseId=course_id).execute()
    coursework = coursework_results.get('courseWork', [])
    for assignment in coursework:
        assignments.append(assignment)
    return assignments

# Fetch submissions for an assignment
def list_submissions(course_id, coursework_id):
    submissions = []
    submissions_results = service.courses().courseWork().studentSubmissions().list(courseId=course_id, courseWorkId=coursework_id).execute()
    submissions = submissions_results.get('studentSubmissions', [])
    return submissions

# List all courses, assignments, and submissions
def get_all_courses_assignments_submissions():
    courses = list_courses()

    if not courses:
        print("No courses found.")
        return

    for course in courses:
        print(f"\nCourse Name: {course['name']}")
        print(f"Course ID: {course['id']}")

        # List assignments for the course
        assignments = list_assignments(course['id'])
        if not assignments:
            print("  No assignments found.")
        else:
            for assignment in assignments:
                print(f"  Assignment: {assignment['title']}")
                print(f"    Assignment ID: {assignment['id']}")

                # List submissions for the assignment
                submissions = list_submissions(course['id'], assignment['id'])
                if not submissions:
                    print("    No submissions found.")
                else:
                    for submission in submissions:
                        print(f"    Submission ID: {submission['id']}")
                        print(f"    Submitted by: {submission['userId']}")
                        print(f"    Grade: {submission.get('grade', 'Not graded yet')}")
                        print(f"    Late: {submission.get('late', 'No')}")

# Run the function to get all data
get_all_courses_assignments_submissions()


HttpError: <HttpError 403 when requesting https://classroom.googleapis.com/v1/courses?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'ACCESS_TOKEN_SCOPE_INSUFFICIENT', 'domain': 'googleapis.com', 'metadata': {'service': 'classroom.googleapis.com', 'method': 'google.classroom.v1.Courses.ListCourses'}}]">

In [ ]:
# Install the client library
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [37]:

# Upload credentials.json (using the file upload button in the left sidebar)

import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

SCOPES = ["https://www.googleapis.com/auth/classroom.courses.readonly"]

def main():
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("classroom", "v1", credentials=creds)
        results = service.courses().list(pageSize=10).execute()
        courses = results.get("courses", [])
        if not courses:
            print("No courses found.")
            return
        print("Courses:")
        for course in courses:
            print(course["name"])
    except HttpError as error:
        print(f"An error occurred: {error}")

if __name__ == "__main__":
    main()

Error: could not locate runnable browser

In [38]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import pickle
import io
import googleapiclient.errors
from googleapiclient.http import MediaIoBaseDownload

# Define the required scopes
SCOPES = [
    "https://www.googleapis.com/auth/classroom.courses",
    "https://www.googleapis.com/auth/classroom.coursework.me",
    "https://www.googleapis.com/auth/classroom.student-submissions.me.readonly",
    "https://www.googleapis.com/auth/classroom.student-submissions.students.readonly",
    "https://www.googleapis.com/auth/classroom.announcements.readonly",
    "https://www.googleapis.com/auth.classroom.profile.emails",
    "https://www.googleapis.com/auth.classroom.profile.photos",
    "https://www.googleapis.com/auth/drive.readonly",  # Added permission for Google Drive
]

def main():
    """Fetch assignments and submissions from all courses."""
    creds = None

    # Check if token.pickle exists for cached credentials
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)

    # If no valid credentials, authenticate the user
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0, include_granted_scopes="true")

        # Save credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)

    # Build the Classroom API service
    service = build("classroom", "v1", credentials=creds)

    # List courses
    results = service.courses().list(pageSize=10).execute()
    courses = results.get("courses", [])

    if not courses:
        print("No courses found.")
    else:
        print("Courses:")
        for course in courses:
            print(f"{course['name']} ({course['id']})")
            fetch_assignments_and_submissions(service, course['id'])

def fetch_assignments_and_submissions(service, course_id):
    """Fetch assignments and submissions for a given course."""

    # List all assignments (coursework) for the course
    coursework_results = service.courses().courseWork().list(courseId=course_id).execute()
    coursework = coursework_results.get("courseWork", [])

    if not coursework:
        print(f"No assignments found for course {course_id}.")
    else:
        print(f"Assignments for course {course_id}:")
        for assignment in coursework:
            print(f"  - {assignment['title']}")
            fetch_submissions(service, course_id, assignment['id'])

def fetch_submissions(service, course_id, assignment_id):
    """Fetch submissions for a given assignment."""

    # List student submissions for the assignment
    submissions_results = service.courses().courseWork().studentSubmissions().list(
        courseId=course_id,
        courseWorkId=assignment_id
    ).execute()

    submissions = submissions_results.get("studentSubmissions", [])

    if not submissions:
        print(f"  No submissions for assignment {assignment_id}.")
    else:
        print(f"  Submissions for assignment {assignment_id}:")
        for submission in submissions:
            print(f"    - {submission['userId']} - {submission['state']}")
            # Download the submission if it has attachments
            if 'attachments' in submission:
                for attachment in submission['attachments']:
                    download_attachment(service, course_id, assignment_id, submission['id'], attachment)

def download_attachment(service, course_id, assignment_id, submission_id, attachment):
    """Download an attachment from a student submission."""

    # Check if the attachment is a Google Drive file
    if 'driveFile' in attachment:
        file_id = attachment['driveFile']['id']
        file_name = attachment['driveFile']['title']

        # Ensure the downloads directory exists
        if not os.path.exists("downloads"):
            os.makedirs("downloads")

        try:
            # Request the file using the file ID
            request = service.files().get_media(fileId=file_id)
            fh = io.FileIO(f"downloads/{file_name}", 'wb')
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                print(f"  Downloaded {file_name} ({int(status.progress() * 100)}%)")
        except googleapiclient.errors.HttpError as error:
            print(f"  An error occurred while downloading {file_name}: {error}")
    else:
        print(f"  No downloadable attachment found for {attachment.get('title', 'Unknown')}.")

if __name__ == "__main__":
    main()


Courses:
Test (743679944986)
Assignments for course 743679944986:
  - Assignment 1
  Submissions for assignment 733607064849:
    - 115932242664249481100 - TURNED_IN
